# 为什么做这个文件
我们小组对郑老师所说的更多内容（如网络评论）感兴趣，同时队伍中我的工作量没有很大，因此做了一个爬虫用以开头

In [4]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import time

def get_html(url):
    '''
    获得 HTML，返回text格式，模拟网页访问，网站会一定程度反爬
    '''
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/53\
        7.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
        #判断网页返回的状态码，200代表连接成功
    else:
        return

def get_infos(html):
    r_xpath = etree.HTML(html)
    title = r_xpath.xpath('//*[@id="articlelistnew"]/div[2]/span[3]/a/text()')
    date = r_xpath.xpath('//*[@id="articlelistnew"]/div[2]/span[5]/text()')  
    datas = []
    datas.append({'date':date,'title':title})
    return datas

# 使用xpath定位内容
股吧的网页与一般教程网页区别较大，我在正则表达式、selector、特征和xpath中走了许多弯路，查了许多教程，各种冲突报错不输出，自己查参数查函数查格式最后终于输出了（泪目！！）
# 函数返回值
`get_html()`返回所给url的text文本格式，`get_infos()`所给text的文本web通过`etree_HTML()`函数输出为xpath的格式，通过定位选取了该网页第一个帖子的date和title，并且打包为字典格式输出。<br>

最初选择字典是易于输出json，但后续发现网页编码和正常编码不一致，输出的json无法打开，在**助教薛凤鸣**师兄的帮助下将字典转化为dataframe格式输出为csv格式（再次泪目！！）

In [5]:
all_datas = []
urls = ['http://guba.eastmoney.com/list,601727,f_{}.html'
            .format(str(i)) for i in range(1, 24)]    
for url in urls:
    html = get_html(url)
    all_datas.extend(get_infos(html))
    time.sleep(1)
    #每过一秒，再次执行下一个网页，避免给服务器造成压力

# 主函数
使用了一个循环，查找了24页的帖子，都是输出第一条帖子的评论和时间，在下面可以看到时间有一定跨度，由于工程量没有进行文本倾向分析的建模，所以只选取了24个帖子作为参考。

In [6]:
all_datas

[{'date': ['08-20 00:12'], 'title': ['跑步入场！投资者数量首破1.7亿！新增2600亿杠杆资金 ']},
 {'date': ['08-18 15:04'], 'title': ['要整体上市吗']},
 {'date': ['08-18 09:05'], 'title': ['上海电气：融资净买入1829.49万元，融资余额8.8亿元（']},
 {'date': ['08-14 21:56'], 'title': ['[惨!关灯吃面][惨!关灯吃面][惨!关灯吃面]']},
 {'date': ['08-13 22:31'], 'title': ['政策性AMC还有多远？']},
 {'date': ['08-12 13:34'], 'title': ['手续费太便宜了']},
 {'date': ['08-11 14:56'], 'title': ['这就不能怪庄家不给力了，架不住猪队友太多']},
 {'date': ['08-11 11:35'], 'title': ['中心国际断工化为了，你们说还能指望谁？']},
 {'date': ['08-08 11:06'], 'title': ['无聊看了一下自选股，全身翻倍股，后悔买了你这垃圾']},
 {'date': ['08-06 21:26'], 'title': ['8月5日，世界品牌大会正式发布2020年《中国500最具价']},
 {'date': ['08-06 10:22'], 'title': ['上海电气第一目标进场位在5.60元左右']},
 {'date': ['08-05 09:29'], 'title': ['两万手压制单']},
 {'date': ['08-04 10:09'], 'title': ['昨天6元解套卖了[鼓掌]']},
 {'date': ['08-03 14:09'], 'title': ['立即买入']},
 {'date': ['07-30 15:20'], 'title': ['天天找跌，缺跌来找我吧！[怒]']},
 {'date': ['07-28 18:43'], 'title': ['明天一字板']},
 {'date': ['07-25 01:13'], 'title': ['下星期准备建仓了，看好5倍。

In [7]:
type(all_datas)

list

In [9]:
import pandas as pd
df = pd.DataFrame(all_datas, columns=['date', 'title']) 
df.head()

,date,title
0,[08-20 00:12],[跑步入场！投资者数量首破1.7亿！新增2600亿杠杆资金 ]
1,[08-18 15:04],[要整体上市吗]
2,[08-18 09:05],[上海电气：融资净买入1829.49万元，融资余额8.8亿元（]
3,[08-14 21:56],[[惨!关灯吃面][惨!关灯吃面][惨!关灯吃面]]
4,[08-13 22:31],[政策性AMC还有多远？]


In [13]:
df.to_csv('D:\个人\软件训练营\Huge-Project\extra\评论.csv',encoding = 'utf-8-sig')